# DistilBERT - Transformers para Reducción de Overfitting

## Objetivo
Implementar DistilBERT (modelo Transformer) como solución de nivel experto para reducir overfitting manteniendo F1-score > 0.55.

## Ventajas de DistilBERT
- ✅✅✅ Entiende contexto y semántica (no solo frecuencias)
- ✅✅✅ Pre-entrenado en millones de textos
- ✅✅✅ Fine-tuning con tu dataset (aprende patrones específicos)
- ✅✅✅ Mejor control de overfitting que modelos clásicos
- ✅✅✅ 60% más rápido que BERT
- ✅✅✅ Dropout incorporado (regularización automática)
- ✅ Captura sutilezas del hate speech que TF-IDF no puede


## 1. Instalación de librerías necesarias


In [10]:
# Instalar si no están instaladas (descomentar si es necesario)
# !pip install transformers torch accelerate

# Suprimir warnings de TensorFlow (no lo necesitamos para DistilBERT)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suprimir warnings de TensorFlow
import warnings
warnings.filterwarnings('ignore')

# Suprimir errores de protobuf/TensorFlow
try:
    import tensorflow as tf
    tf.get_logger().setLevel('ERROR')
except:
    pass

import pandas as pd
import numpy as np
import pickle
import random
from pathlib import Path
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import (
    DistilBertTokenizer, DistilBertForSequenceClassification,
    get_linear_schedule_with_warmup
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix
)

# Configurar dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Dispositivo: {device}")

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

print("✅ Librerías importadas (enfoque directo con PyTorch, sin datasets)")


✅ Dispositivo: cpu
✅ Librerías importadas (enfoque directo con PyTorch, sin datasets)


## 2. Carga de datos


In [11]:
# Cargar datos
df = pd.read_csv('../data/processed/youtoxic_english_1000_processed.csv')
with open('../data/processed/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)
with open('../data/processed/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

# Para Transformers, usar texto original o preprocesado (probamos con preprocesado)
X_train_text = df[df.index.isin(range(len(y_train)))]['Text_processed'].values
X_test_text = df[df.index.isin(range(len(y_train), len(y_train) + len(y_test)))]['Text_processed'].values

print(f"✅ Datos cargados: {len(X_train_text)} train, {len(X_test_text)} test")
print(f"Distribución train: {np.bincount(y_train)}")
print(f"Distribución test: {np.bincount(y_test)}")


✅ Datos cargados: 800 train, 200 test
Distribución train: [430 370]
Distribución test: [108  92]


## 3. Cargar tokenizador y modelo


In [12]:
# Cargar tokenizador y modelo pre-entrenado
model_name = 'distilbert-base-uncased'

print(f"Cargando {model_name}...")
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  # Clasificación binaria
    problem_type="single_label_classification"
)

print(f"✅ Modelo cargado: {model_name}")
print(f"   Parámetros: {model.num_parameters():,}")


Cargando distilbert-base-uncased...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Modelo cargado: distilbert-base-uncased
   Parámetros: 66,955,010


## 4. Dataset personalizado para PyTorch


In [13]:
class HateSpeechDataset(Dataset):
    """Dataset personalizado para clasificación de hate speech."""
    
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])
        
        # Tokenizar texto
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

print("✅ Clase Dataset definida")


✅ Clase Dataset definida


## 5. Función de evaluación


In [14]:
def evaluate_model(model, dataloader, device):
    """Evalúa el modelo y retorna métricas."""
    model.eval()
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)
    
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, zero_division=0)
    recall = recall_score(true_labels, predictions, zero_division=0)
    f1 = f1_score(true_labels, predictions, zero_division=0)
    cm = confusion_matrix(true_labels, predictions)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': cm,
        'predictions': predictions
    }

print("✅ Función de evaluación definida")


✅ Función de evaluación definida


## 6. Preparar datasets


In [15]:
# Crear datasets
train_dataset = HateSpeechDataset(X_train_text, y_train, tokenizer)
test_dataset = HateSpeechDataset(X_test_text, y_test, tokenizer)

print(f"✅ Datasets creados")
print(f"   Train: {len(train_dataset)} ejemplos")
print(f"   Test: {len(test_dataset)} ejemplos")


✅ Datasets creados
   Train: 800 ejemplos
   Test: 200 ejemplos


## 7. Configurar entrenamiento con anti-overfitting


In [16]:
# Configurar hiperparámetros de entrenamiento
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_EPOCHS = 5
WEIGHT_DECAY = 0.01
WARMUP_STEPS = 100

print("="*80)
print("CONFIGURACIÓN DE ENTRENAMIENTO - DISTILBERT")
print("="*80)
print(f"✅ Épocas: {NUM_EPOCHS}")
print(f"✅ Learning rate: {LEARNING_RATE}")
print(f"✅ Weight decay: {WEIGHT_DECAY} (regularización L2)")
print(f"✅ Batch size: {BATCH_SIZE}")
print(f"✅ Warmup steps: {WARMUP_STEPS}")
print(f"✅ Early stopping: Sí (patience=2)")
print("-"*80)


CONFIGURACIÓN DE ENTRENAMIENTO - DISTILBERT
✅ Épocas: 5
✅ Learning rate: 2e-05
✅ Weight decay: 0.01 (regularización L2)
✅ Batch size: 16
✅ Warmup steps: 100
✅ Early stopping: Sí (patience=2)
--------------------------------------------------------------------------------


## 8. Entrenar modelo (enfoque directo con PyTorch)


In [17]:
# Crear DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Mover modelo a dispositivo
model = model.to(device)

# Configurar optimizador y scheduler
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
total_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=total_steps
)

# Función de pérdida
criterion = nn.CrossEntropyLoss()

print("="*80)
print("ENTRENAMIENTO DISTILBERT - CONTROL DE OVERFITTING")
print("="*80)
print("✅ Early stopping (patience=2)")
print("✅ Weight decay (regularización L2)")
print("✅ Learning rate bajo (2e-5)")
print("✅ Pocas épocas (5)")
print("✅ Evaluación cada época")
print("-"*80)

# Entrenar modelo
best_f1 = 0
patience_counter = 0
patience = 2

for epoch in range(NUM_EPOCHS):
    # Entrenamiento
    model.train()
    total_loss = 0
    
    progress_bar = tqdm(train_loader, desc=f'Época {epoch+1}/{NUM_EPOCHS}')
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})
    
    avg_loss = total_loss / len(train_loader)
    
    # Evaluación
    eval_results = evaluate_model(model, test_loader, device)
    test_f1 = eval_results['f1']
    
    print(f"\nÉpoca {epoch+1}/{NUM_EPOCHS}")
    print(f"  Loss: {avg_loss:.4f}")
    print(f"  F1 (test): {test_f1:.4f}")
    print(f"  Accuracy (test): {eval_results['accuracy']:.4f}")
    
    # Early stopping
    if test_f1 > best_f1:
        best_f1 = test_f1
        patience_counter = 0
        # Guardar mejor modelo
        os.makedirs('../models', exist_ok=True)
        torch.save(model.state_dict(), '../models/distilbert_best_model.pt')
        print(f"  ✅ Mejor modelo guardado (F1: {best_f1:.4f})")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"  ⏹️  Early stopping (patience={patience})")
            break

# Cargar mejor modelo
model.load_state_dict(torch.load('../models/distilbert_best_model.pt'))
print("\n✅ Entrenamiento completado")


ENTRENAMIENTO DISTILBERT - CONTROL DE OVERFITTING
✅ Early stopping (patience=2)
✅ Weight decay (regularización L2)
✅ Learning rate bajo (2e-5)
✅ Pocas épocas (5)
✅ Evaluación cada época
--------------------------------------------------------------------------------


Época 1/5:   0%|                                                                                                         | 0/50 [00:00<?, ?it/s]


KeyError: 493

## 9. Evaluación del modelo


In [ ]:
# Evaluar en train y test para calcular overfitting
print("Evaluando en conjunto de entrenamiento...")
train_results = evaluate_model(model, train_loader, device)

print("\nEvaluando en conjunto de prueba...")
test_results = evaluate_model(model, test_loader, device)

# Calcular diferencia F1 (overfitting)
diff_f1 = abs(train_results['f1'] - test_results['f1']) * 100

print("\n" + "="*80)
print("RESULTADOS FINALES - DISTILBERT")
print("="*80)
print(f"F1-score (train): {train_results['f1']:.4f}")
print(f"F1-score (test): {test_results['f1']:.4f}")
print(f"Accuracy (test): {test_results['accuracy']:.4f}")
print(f"Precision (test): {test_results['precision']:.4f}")
print(f"Recall (test): {test_results['recall']:.4f}")
print(f"Diferencia F1: {diff_f1:.2f}%")
print(f"\nMatriz de confusión (test):")
print(test_results['confusion_matrix'])

if diff_f1 < 5.0 and test_results['f1'] > 0.55:
    print("\n✅✅✅ OBJETIVO CUMPLIDO: Overfitting < 5% Y F1 > 0.55")
elif diff_f1 < 6.0:
    print("\n🎯 MUY CERCA: Overfitting < 6%")
else:
    print("\n⚠️  Overfitting aún alto")

print("="*80)


## 10. Guardar modelo (si cumple objetivos)


In [ ]:
if diff_f1 < 6.0 and test_results['f1'] > 0.55:
    # Guardar modelo y tokenizador
    model_path = Path('../models/distilbert_model')
    model_path.mkdir(parents=True, exist_ok=True)
    
    # Guardar modelo completo
    model.save_pretrained(str(model_path))
    tokenizer.save_pretrained(str(model_path))
    
    # Guardar información del modelo
    model_info = {
        'model_type': 'DistilBERT',
        'model_name': model_name,
        'test_f1': test_results['f1'],
        'diff_f1': diff_f1,
        'train_f1': train_results['f1'],
        'num_epochs': NUM_EPOCHS,
        'learning_rate': LEARNING_RATE,
        'weight_decay': WEIGHT_DECAY,
        'batch_size': BATCH_SIZE
    }
    
    with open('../models/distilbert_info.pkl', 'wb') as f:
        pickle.dump(model_info, f)
    
    print(f"\n✅ Modelo guardado en: {model_path}")
    print(f"✅ Información guardada en: ../models/distilbert_info.pkl")
else:
    print("\n⚠️  Modelo no guardado (no cumple objetivos)")
    print(f"   Overfitting: {diff_f1:.2f}% (objetivo: <6%)")
    print(f"   F1-score: {test_results['f1']:.4f} (objetivo: >0.55)")
